In [57]:
!pip install -q pyomo

In [58]:
from pyomo.environ import *
import numpy as np

***Question1,2,3***

max $0.65x_1$ + $0.75x_2$ − $5.85x_3$ − $8.35x_4$ − $12.38x_5$ + $30x_6$

s.t.

 −$0.5x_1$ − $3.8x_3$ − $0.9x_4$ + $0.9x_5$ ≤ 24.8

$x_1$ − $x_4$ − $x_6$ ≤ 0.8

−$2.9x_2$ − $1.9x_3$ − $0.9x_6$ ≤ −71.4

$4.7x_4$ + $8.02x_5$ + $6.15x_6$ ≤ 95

−$0.7x_1$ − $1.4x_2$ + $7.9x_3$ − $2.9x_4$ + $0.9x_5$ ≤ −47

$x_i$ ≥ 0 ∀i ∈ {1, 2, . . . , 6}.

Creating text file using notepad and and copied to the colab environment

In [59]:
coef = np.loadtxt('lab4_ex1_coef.txt',delimiter = ',')#Question4 loading coef file using numpy.loadtxt

In [60]:
print(coef)

[[  0.65   0.75  -5.85  -8.35 -12.38  30.     0.  ]
 [ -0.5    0.    -3.8   -0.9    0.9    0.    24.8 ]
 [  1.     0.     0.    -1.     0.    -1.     0.8 ]
 [  0.    -2.9   -1.9    0.     0.    -0.9  -71.4 ]
 [  0.     0.     0.     4.7    8.02   6.15  95.  ]
 [ -0.7   -1.4    7.9   -2.9    0.9    0.   -47.  ]]


In [61]:
coef.shape
print("Num rows   :",coef.shape[0])
print("Num columns:",coef.shape[1])

Num rows   : 6
Num columns: 7


In [62]:
model = ConcreteModel()

In [63]:
N = coef.shape[1]-1 # No of variables(removing rhs column)
M = coef.shape[0]-1 # No of constraints(removing objective row)

In [64]:
obj_coef = coef[0,:-1]
print(obj_coef)

[  0.65   0.75  -5.85  -8.35 -12.38  30.  ]


In [65]:
constr_coef = coef[1:,:-1]
print(constr_coef)

[[-0.5   0.   -3.8  -0.9   0.9   0.  ]
 [ 1.    0.    0.   -1.    0.   -1.  ]
 [ 0.   -2.9  -1.9   0.    0.   -0.9 ]
 [ 0.    0.    0.    4.7   8.02  6.15]
 [-0.7  -1.4   7.9  -2.9   0.9   0.  ]]


In [66]:
constr_rhs = coef[1:,-1]
print(constr_rhs)

[ 24.8   0.8 -71.4  95.  -47. ]


In [67]:
lower_bound = 0
upper_bound = np.inf

In [68]:
col_indices = np.arange(N)
row_indices = np.arange(M)
print(col_indices)
print(row_indices)

[0 1 2 3 4 5]
[0 1 2 3 4]


In [69]:
model.x = Var(col_indices)

In [70]:
model.constraints = ConstraintList()

In [71]:
for i in row_indices:
  model.constraints.add(summation(constr_coef[i],model.x)<=constr_rhs[i])

In [72]:
for j in col_indices:
  model.x[j].setlb(lower_bound)
  model.x[j].setub(upper_bound)

In [73]:
model.objective = Objective(expr = summation(obj_coef,model.x),sense = maximize)

In [74]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals
          5 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.65*x[0] + 0.75*x[1] - 5.85*x[2

In [75]:
!apt-get install -y -qq coinor-cbc

In [76]:
opt_cbc = SolverFactory('cbc')

***Question 5&6***

LPP is created and using cbc solver to solve the model

In [77]:
result = opt_cbc.solve(model)
print('Solver Status: ',result.solver.status)
print('Solver termination condition: ',result.solver.termination_condition)

  - termination condition: unbounded
  - message from solver: <undefined>


Solver Status:  warning
Solver termination condition:  unbounded


***Question 7,8,9***

Results obtained from cbc solver :
 * Solver Status:  warning
 * Solver termination condition:  unbounded

 Since LPP is maximization and termination condition is unbounded so objective function will have value inf and each variable take infinte as value. So if we give some upper bound to each variable we can get some optimal solution according to bounds.


 Looking at the constraint $4.7x_4$ + $8.02x_5$ + $6.15x_6$ ≤ 95

 the upper bound is 95 and coefficients is +ve so if we take $x_4$ = $x_5$ = $x_6$ we get value of x around 5  So lets take an upper bound of 50 for each variable

In [78]:
for i in col_indices:
  model.x[i].setub(50)

In [79]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :    50 : False :  True :  Reals
          1 :     0 :  None :    50 : False :  True :  Reals
          2 :     0 :  None :    50 : False :  True :  Reals
          3 :     0 :  None :    50 : False :  True :  Reals
          4 :     0 :  None :    50 : False :  True :  Reals
          5 :     0 :  None :    50 : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.65*x[0] + 0.75*x[1] - 5.85*x[2

In [80]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [81]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()


Objective =  511.4752701

Decision Variables
x[ 0 ] =  16.247154
x[ 1 ] =  50.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0
x[ 5 ] =  15.447154

Constraints
constraints : Size=5
    Key : Lower : Body               : Upper
      1 :  None :          -8.123577 :  24.8
      2 :  None : 0.7999999999999989 :   0.8
      3 :  None :       -158.9024386 : -71.4
      4 :  None :         94.9999971 :  95.0
      5 :  None :        -81.3730078 : -47.0


***Question9***

Solver status after modification of upper bounds 

* Solver status: ok
* Solver termination condition: optimal

Since we got the optimal solution and values of varaibles and objective function for the given upper bound is 


Objective_value =  511.4752701

Decision Variables
* $x_1$=  16.247154
* $x_2$ =  50.0
* $x_3$ =  0.0
* $x_4$ =  0.0
* $x_5$ =  0.0
* $x_6$ =  15.447154

Constraints Active Status:

* Constraint_1 is Not Active as LHS != RHS
* Constraint_2 is Active     as LHS  = RHS
* Constraint_3 is Not Active as LHS != RHS
* Constraint_4 is Active     as LHS  = RHS
* Constraint_5 is Not Active as LHS != RHS

